<a href="https://colab.research.google.com/github/arumajirou/-daily-test/blob/main/bta_lib_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# bta-libとは

---
**bta-libは、「backtrader ta-lib」（すなわち、「テクニカル分析ライブラリ」）の略称です。その名の通り、backtraderファミリーの一部です。**

---
新しい指標を開発・実験するために、ユーザビリティ、再利用性、使いやすさに重点を置いたpandasベースのライブラリです。



# Yet Another Technical Analysis LIBrary

---
もうひとつのテクニカル分析LIBrary

---




- 使いやすく、再利用しやすいライブラリーを持つこと

- インジケーターの正しい実装を提案

- 読みやすい指標の実装を実現する

- 新しい指標を迅速かつ容易に開発することを可能にする。

# デザイン
最終的に選ばれた道は、コンポジションとラッピングを混ぜることで、pandas.Seriesをバックグラウンドで管理しながら、pandas.Seriesが提供するAPIをほぼ直接かつ透過的に使って計算を行うことができるようにすることでした

---



pandas.SeriesのAPIを使用することは、基本的な指標を開発するときにのみ、通常明確に見ることができます。

---


\
**例えば、こんな感じです**。

DEMAはこのような式になります。2 * EMA(データ、期間) - EMA(EMA(データ、期間))

その式では、pandas.SeriesAPIがどこにあるかは明らかではありません。


# 考慮すべきこと



1.   最小サポートバージョンはPython 3.6です。
2.   pandasは依存関係にあります。

つまり、pip install bta-libを実行すると、途中でpandas（と関連するすべての依存関係）をインストールしようとします。

特にWindowsで、CPythonの標準配布を使う場合、OS標準のコンパイラがないため、長く、退屈で、つらい作業になることがあります。



In [1]:
!pip install bta-lib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 92 kB 38 kB/s 


In [9]:
import btalib
import pandas as pd

# csvファイルをpandasのデータフレームに読み込む
txt_path ="https://raw.githubusercontent.com/mementum/bta-lib/master/data/2006-day-001.txt"
df = pd.read_csv(txt_path, parse_dates=True, index_col='Date')
df

,Open,High,Low,Close,Volume,OpenInterest
Date,,,,,,
2006-01-02,1789.36,1802.98,1789.36,1802.16,1647940,0.0
2006-01-03,1802.04,1819.21,1800.92,1807.17,5544260,0.0
2006-01-04,1807.62,1826.23,1807.62,1826.23,5175580,0.0
2006-01-05,1826.10,1830.82,1821.58,1825.12,4322520,0.0
2006-01-06,1825.27,1833.50,1823.83,1833.50,4161120,0.0
...,...,...,...,...,...,...
2006-12-21,2055.93,2062.64,2052.23,2056.05,3746080,0.0
2006-12-22,2054.93,2054.93,2036.31,2036.75,3783530,0.0
2006-12-27,2039.85,2067.43,2039.85,2067.43,3110030,0.0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 255 entries, 2006-01-02 to 2006-12-29
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          255 non-null    float64
 1   High          255 non-null    float64
 2   Low           255 non-null    float64
 3   Close         255 non-null    float64
 4   Volume        255 non-null    int64  
 5   OpenInterest  255 non-null    float64
dtypes: float64(5), int64(1)
memory usage: 13.9 KB


In [23]:
df.size

1530

In [24]:
df.shape

(255, 6)

In [25]:
df.describe

<bound method NDFrame.describe of                Open     High      Low    Close   Volume  OpenInterest
Date                                                                 
2006-01-02  1789.36  1802.98  1789.36  1802.16  1647940           0.0
2006-01-03  1802.04  1819.21  1800.92  1807.17  5544260           0.0
2006-01-04  1807.62  1826.23  1807.62  1826.23  5175580           0.0
2006-01-05  1826.10  1830.82  1821.58  1825.12  4322520           0.0
2006-01-06  1825.27  1833.50  1823.83  1833.50  4161120           0.0
...             ...      ...      ...      ...      ...           ...
2006-12-21  2055.93  2062.64  2052.23  2056.05  3746080           0.0
2006-12-22  2054.93  2054.93  2036.31  2036.75  3783530           0.0
2006-12-27  2039.85  2067.43  2039.85  2067.43  3110030           0.0
2006-12-28  2068.72  2071.03  2062.57  2065.33  2822950           0.0
2006-12-29  2065.06  2071.00  2059.97  2059.97  2058160           0.0

[255 rows x 6 columns]>

In [26]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'OpenInterest'], dtype='object')

In [18]:
#SMA（単純移動平均）を計算する
sma = btalib.sma(df)
print(sma)

手始めにデータファイルの最初の2行のサンプルを見てみましょう。このファイルは株式市場の資産としては非常に一般的なフォーマットになっています。

In [17]:
df[:1]

,Open,High,Low,Close,Volume,OpenInterest
Date,,,,,,
2006-01-02,1789.36,1802.98,1789.36,1802.16,1647940,0.0


SMAは、Close欄の値を用いて算出した。

In [19]:
sma = btalib.sma(df.High)
sma

sma = btalib.sma(df)から得られるデフォルト値は、ライブラリ自体の内部 オブジェクトです。その理由は、他の指標で追加計算をするために使われる可能性があるからです。しかし、マーケットで使えるものを手に入れるのは簡単です。
ということです。結果のDataFrameを取得するのはとても簡単です。ワンライナーでも十分です。

In [13]:
sma = btalib.sma(df).df  # デフォルトの周期は30
sma

,sma
Date,
2006-01-02,NaN
2006-01-03,NaN
2006-01-04,NaN
2006-01-05,NaN
2006-01-06,NaN
...,...
2006-12-21,2029.800333
2006-12-22,2029.961333
2006-12-27,2030.773333


In [21]:
df.Close

Date
2006-01-02    1802.16
2006-01-03    1807.17
2006-01-04    1826.23
2006-01-05    1825.12
2006-01-06    1833.50
               ...   
2006-12-21    2056.05
2006-12-22    2036.75
2006-12-27    2067.43
2006-12-28    2065.33
2006-12-29    2059.97
Name: Close, Length: 255, dtype: float64

しかし、多くの場合、2ライナーの方が読みやすく、情報もよく伝わります。print(sma_df)の結果を見てみましょう。

In [14]:
sma = btalib.sma(df, period=4)  # デフォルトの周期は30で、4へ変更
print(sma.df)

                  sma
Date                 
2006-01-02        NaN
2006-01-03        NaN
2006-01-04        NaN
2006-01-05  1815.1700
2006-01-06  1823.0050
...               ...
2006-12-21  2057.6475
2006-12-22  2050.5775
2006-12-27  2054.8750
2006-12-28  2056.3900
2006-12-29  2057.3700

[255 rows x 1 columns]


# より深く掘り下げるパラメータ
このコードには、「デフォルトの周期は30です」というコメントが含まれています。そして、初期値がNaN（"Not a Number"）である結果を見れば、コメントに書かれていることは明らかでしょう。

- **スマが値を出し始めるには、30個のデータポイントが必要です。**

したがって，最初のNaNは，意味のある値が計算され配信されないことを示しています．念のため，print(sma.df.to_string())の抜粋で，smaの計算が値を出し始めたときの様子を示しています．

In [15]:
sma = btalib.sma(df.High, period=4)  # デフォルトの周期は30で、4へ変更
print(sma.df)

                  sma
Date                 
2006-01-02        NaN
2006-01-03        NaN
2006-01-04        NaN
2006-01-05  1819.8100
2006-01-06  1827.4400
...               ...
2006-12-21  2064.8175
2006-12-22  2060.8675
2006-12-27  2062.6000
2006-12-28  2064.0075
2006-12-29  2066.0975

[255 rows x 1 columns]


# どの列を使うかは列を渡すことで指定する


---

このインジケータはたった1つの入力を必要とします。具体的には、df.High(Series)を渡すと、そのデータフィールドで直接計算が実行されます。

In [ ]:
sma = btalib.sma(df.High)

# smaオブジェクトを再利用し、自分自身に渡す
---
smaは出力も1つです。したがって、それ自身のような別の指標の単一の入力として直接使用することができます...。大したことはない（sma1の最初の結果の配信期間は明らかに増加する）。



In [27]:
sma = btalib.sma(df)
sma1 = btalib.sma(sma)
print(sma)
print(sma1)

# リブレイのaagicにインデックスで列を探させる

---
おっと!Close の名前をNewClose に変更することで、列の名前はもはや見つからず、他の列はsma が求める入力の名前と一致しない。上で説明したように、そして標準的なOHLCVOi の順序に従って、Closeは3のインデックスを持ち、そのインデックスに存在する列が選ばれる。

データフレームには2つの列（とインデックス）しかなく、したがって、インデックス0と 1しか利用できないことがあります。この場合、機械はデフォルトで最初の列、すなわち列0を使用します。

これは、インジケータが単一入力を期待しており、データフレーム形式の単一入力が提供されているため、妥当な仮定と選択と見なされる。名前マッチングとデフォルトの列インデックスマッチングの両方が失敗した場合、単一入力のデータフレームの最初の列が選択される。



In [29]:
df.rename(columns={'Close':'NewClose'}, inplace=True)
sma = btalib.sma(df)
print(sma)